In [ ]:
!pip install --quiet google-api-python-client isodate

In [ ]:
from googleapiclient.discovery import build
from isodate import parse_duration
import pandas as pd
import time

API_KEY = ""
youtube = build("youtube", "v3", developerKey=API_KEY)

In [ ]:
# 日付範囲をISO形式で指定（例：2025年7月3日〜14日）
published_after = "2025-07-03T00:00:00Z"
published_before = "2025-07-19T23:59:59Z"

In [ ]:
def get_videos_from_channel_by_keyword(channel_id, keyword, published_after, published_before):
    video_ids = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part="id",
            channelId=channel_id,
            q=keyword,
            type="video",
            maxResults=50,
            publishedAfter=published_after,
            publishedBefore=published_before,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response.get("items", []):
            video_ids.append(item["id"]["videoId"])

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break
        time.sleep(0.1)

    return get_video_details(video_ids)


def get_video_details(video_ids):
    all_data = []
    for i in range(0, len(video_ids), 50):
        response = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(video_ids[i:i+50])
        ).execute()

        for item in response['items']:
            try:
                duration_sec = parse_duration(item['contentDetails']['duration']).total_seconds()
            except:
                duration_sec = None

            data = {
                'videoId': item['id'],
                'title': item['snippet']['title'],
                'description': item['snippet']['description'],
                'channelTitle': item['snippet']['channelTitle'],
                'publishedAt': item['snippet']['publishedAt'],
                'viewCount': int(item['statistics'].get('viewCount', 0)),
                'duration': duration_sec
            }
            all_data.append(data)
        time.sleep(0.2)
    return all_data


In [ ]:
keywords_kdp = ["国民民主党", "国民民主", "玉木"]
keywords_ssp = ["参政党", "参政", "神谷"]

In [ ]:
group1 = [
    "UCuTAXTexrhetbOe3zgskJBQ",  # 日テレ
    "UC6AG81pAkf6Lbi_1VC5NmPA",  # TBS
    "UCoQBJMzcwmXrRSHBFAlTsIw",  # FNN
    "UCGCZAYq5Xxojl_tSXcVJhiQ",  # ANN
    "UCB1dgsqLiEp57oDAyNV_vww",  # ABEMA
    "UC28hcmMG3TJv60Byo528VVg",  # ニコニコ
    "UCG_oqDSlIYEspNpd2H4zWhw",  # ReHacQ
]

group2 = [
    "UCwywumWHI4_hQtZpJwsC8cw",  # 高須幹弥
    "UC3SMMsAG5lFTBlvgsTH5Z_Q",  # 政治家ターゲット
    "UC1I-Bsl41dhgyArBVXjGU_w",  # ココノセイジ
    "UCHZ8x1slRhaszSrydy9En_Q",  # JAPANの事実
    "UCNJG9cFS3hHZtp4JXtnpc5A",  # 日本政治TIME
    "UCk5ADs9g7nim-75LsY1sxVA",  # センキョタイムズ
]

group3 = [
    "UCb_oJ33zleCwX683QmYj4WA",  # やまとの政治塾
    "UC0aToF412eWBjYJ6pPzUuZQ",  # 井川意高
    "UCDV545x0n25UbV5vslyxohA",  # Pome Scandal
    "UCoo8X9PibPKHIWKH6OfXj_w",  # 政治師たち
    "UCh4r6eRHM8Ux7LEbCQF4svQ",  # 初心政治3
    "UCLpDf-oT5i-qQFmNFFbDYkw",  # 人類予備校
]

group4 = [
    "UCBKn5J5o01uBYzwuC7HnUbQ",  # 撮って出しニュース
    "UCWBNxVkMmRst6CubagtP7Sw",  # SOOC News
    "UCQG3UwelmcEiPrQCp5GzOoQ",  # Hot News
    "UC7dXPammhcS6lrPzgq7WAPQ",  # 文化人放送局
]


In [ ]:
videos_group1 = []
for cid in group1:
    for kw in keywords_kdp + keywords_ssp:
        videos_group1 += get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
print(f"Group1 total videos: {len(videos_group1)}")


Group1 total videos: 1551


In [ ]:
videos_group2 = []
for cid in group2:
    for kw in keywords_kdp + keywords_ssp:
        videos_group2 += get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
print(f"Group2 total videos: {len(videos_group2)}")


Group2 total videos: 868


In [ ]:
# 国民民主党（KDP）関連動画
videos_group3_kdp = []
for cid in group3:
    for kw in keywords_kdp:
        results = get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
        for video in results:
            video["source"] = "kdp"
        videos_group3_kdp += results
print(f"Group3 (KDP) total videos: {len(videos_group3_kdp)}")

# 参政党（SSP）関連動画
videos_group3_ssp = []
for cid in group3:
    for kw in keywords_ssp:
        results = get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
        for video in results:
            video["source"] = "ssp"
        videos_group3_ssp += results
print(f"Group3 (SSP) total videos: {len(videos_group3_ssp)}")


Group3 (KDP) total videos: 81
Group3 (SSP) total videos: 497


In [ ]:
# 国民民主党（KDP）関連動画 - Group4
videos_group4_kdp = []
for cid in group4:
    for kw in keywords_kdp:
        results = get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
        for video in results:
            video["source"] = "kdp"
        videos_group4_kdp += results
print(f"Group4 (KDP) total videos: {len(videos_group4_kdp)}")

# 参政党（SSP）関連動画 - Group4
videos_group4_ssp = []
for cid in group4:
    for kw in keywords_ssp:
        results = get_videos_from_channel_by_keyword(cid, kw, published_after, published_before)
        for video in results:
            video["source"] = "ssp"
        videos_group4_ssp += results
print(f"Group4 (SSP) total videos: {len(videos_group4_ssp)}")


Group4 (KDP) total videos: 121
Group4 (SSP) total videos: 162


In [ ]:
import pandas as pd

# videos_group1, videos_group2 がすでに存在している前提
combined_videos = videos_group1 + videos_group2

# DataFrame に変換
df_videos = pd.DataFrame(combined_videos)

# 件数とプレビューを表示
print(f"Total videos in group1 + group2: {len(df_videos)}")
display(df_videos.head())


Total videos in group1 + group2: 2419


,videoId,title,description,channelTitle,publishedAt,viewCount,duration
0,OdvxFQ70yQk,【参院選】国民民主党・玉木代表“初日の訴え”,参議院選挙は７月３日に公示され、20日に投開票が行われます。 選挙戦初日、国民民主党・玉木雄...,日テレNEWS,2025-07-03T02:27:05Z,45597,1092.0
1,0Qu71-ydn0c,【ノーカット】藤井貴彦が党首にきく！国民民主党 玉木雄一郎代表,参院選を「皆さんの手取りを増やす夏」と表現した、国民民主党、玉木雄一郎代表。「手取りを増やす...,日テレNEWS,2025-07-10T14:59:06Z,549724,1954.0
2,ZHsypI_uyI4,【参院選】主要8党首×藤井キャスター 「再生回数」最多の政党は？ コメント数・年代…ノーカッ...,20日投開票の参議院選挙を前に、藤井貴彦キャスターはこれまで、主要8党の党首への個別インタビ...,日テレNEWS,2025-07-19T10:13:59Z,113888,292.0
3,Eiq_GI7Brvw,【与党】過半数維持は微妙な情勢 国民民主・参政が大幅議席増の勢い NNN・読売新聞参院選情勢調査,NNNは参議院選挙の序盤の情勢を分析しました。その結果、自民党と公明党の与党が非改選議席を合...,日テレNEWS,2025-07-05T04:19:38Z,111968,144.0
4,RvO4v7X1Mtg,【参議院選挙】与党過半数確保「厳しい情勢」 国民民主・参政は勢い維持,NNNは参議院選挙の終盤の情勢を分析しました。自民党と公明党の与党は序盤より情勢が悪化し、非...,日テレNEWS,2025-07-15T23:51:53Z,241136,144.0


In [ ]:
# UTF-8 BOM付きでCSV保存（Excelで文字化けしない）
df_videos.to_csv("youtube_group1_group2.csv", index=False, encoding="utf-8-sig")


In [ ]:
from google.colab import files
files.download("youtube_group1_group2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(df_all.columns)


Index(['videoId', 'title', 'description', 'channelTitle', 'publishedAt',
       'viewCount', 'duration', 'source'],
      dtype='object')


In [ ]:
import pandas as pd

# 1. 各リストを結合
all_videos = videos_group3_kdp + videos_group3_ssp + videos_group4_kdp + videos_group4_ssp

# 2. データフレームに変換
df_all = pd.DataFrame(all_videos)

# 3. videoId で重複を削除
df_all = df_all.drop_duplicates(subset="videoId")

# 4. 件数を表示
print(f"重複排除後の動画本数: {len(df_all)}")

# 5. 必要に応じて CSV に保存（文字化け対策済）
df_all.to_csv("merged_videos.csv", index=False, encoding="utf-8-sig")


重複排除後の動画本数: 325


In [ ]:
from google.colab import files
files.download("merged_videos.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# 各グループの動画情報をDataFrameに変換
df_group3_kdp = pd.DataFrame(videos_group3_kdp)
df_group3_ssp = pd.DataFrame(videos_group3_ssp)
df_group4_kdp = pd.DataFrame(videos_group4_kdp)
df_group4_ssp = pd.DataFrame(videos_group4_ssp)

# 各データセットの先頭5行を表示
print("=== Group3 (KDP) ===")
display(df_group3_kdp.head())

print("=== Group3 (SSP) ===")
display(df_group3_ssp.head())

print("=== Group4 (KDP) ===")
display(df_group4_kdp.head())

print("=== Group4 (SSP) ===")
display(df_group4_ssp.head())


=== Group3 (KDP) ===


,videoId,title,description,channelTitle,publishedAt,viewCount,duration,source
0,krus_ndPfqo,【さとうさおり】※消される前に見てください…国民民主党議員のエグすぎる事実が判明しました…【...,あなたは国民民主党の田中議員についてどう思いますか？\nみなさんの意見をコメント欄で教えてく...,やまとの政治塾,2025-07-05T01:54:23Z,76264,2031.0,kdp
1,KhHBC5nJB_c,【原口一博】※消される前に見てください…恐ろしい事態が発生しました…【自民党/石破茂/小泉進...,あなたは今回の参議院選挙についてどう思いますか？\nみなさんの意見をコメント欄で教えてくださ...,やまとの政治塾,2025-07-19T01:25:16Z,69555,1400.0,kdp
2,krus_ndPfqo,【さとうさおり】※消される前に見てください…国民民主党議員のエグすぎる事実が判明しました…【...,あなたは国民民主党の田中議員についてどう思いますか？\nみなさんの意見をコメント欄で教えてく...,やまとの政治塾,2025-07-05T01:54:23Z,76264,2031.0,kdp
3,HMQJnk9JUw0,【選挙特番】神谷だけは信用できない。参院選に向けて保守政党を語ります。,00:00 はじめに\n00:40 参院選に関するXでのポスト\n04:58 自民・公明・立...,井川意高が熔ける日本を斬る,2025-07-15T11:00:25Z,728391,944.0,kdp
4,HMQJnk9JUw0,【選挙特番】神谷だけは信用できない。参院選に向けて保守政党を語ります。,00:00 はじめに\n00:40 参院選に関するXでのポスト\n04:58 自民・公明・立...,井川意高が熔ける日本を斬る,2025-07-15T11:00:25Z,728391,944.0,kdp


=== Group3 (SSP) ===


,videoId,title,description,channelTitle,publishedAt,viewCount,duration,source
0,ZJdr2OcOyfQ,【この後、拍手喝采】こんな演説見たことない！思わず涙がこぼれる神谷宗幣氏の圧巻スピーチ【参政...,あなたは参政党や神谷宗幣議員についてどう思いますか？\nみなさんの意見をコメント欄で教えてく...,やまとの政治塾,2025-07-05T05:20:56Z,459748,1308.0,ssp
1,5oPR4qXODpU,【拍手喝采】こんな演説見たことない！思わず涙があふれる参政党さや氏の圧巻スピーチ【神谷宗幣/...,あなたは参政党やさや氏についてどう思いますか？\nみなさんの意見をコメント欄で教えてください...,やまとの政治塾,2025-07-09T11:00:27Z,497145,1618.0,ssp
2,Hb_-MYGzgUc,【浜田聡】※大至急見てください…トンデモない事態が発生しました…【参政党/神谷宗幣/三橋貴明...,あなたはTBSの報道特集や山本恵里伽アナについてどう思いますか？\nみなさんの意見をコメント...,やまとの政治塾,2025-07-16T02:38:21Z,1279123,1131.0,ssp
3,1vTGk5OPL-s,【梅村みずほ】※消される前に見てください...トンデモない事態が発生しました【参政党/神谷宗...,あなたは参政党や梅村みずほ議員についてどう思いますか？\nみなさんの意見をコメント欄で教えて...,やまとの政治塾,2025-07-14T09:10:10Z,164278,1473.0,ssp
4,ZJdr2OcOyfQ,【この後、拍手喝采】こんな演説見たことない！思わず涙がこぼれる神谷宗幣氏の圧巻スピーチ【参政...,あなたは参政党や神谷宗幣議員についてどう思いますか？\nみなさんの意見をコメント欄で教えてく...,やまとの政治塾,2025-07-05T05:20:56Z,459748,1308.0,ssp


=== Group4 (KDP) ===


,videoId,title,description,channelTitle,publishedAt,viewCount,duration,source
0,4zJOu26EaAg,【参院選】大敗確定の自民は内部分裂！選挙後の連立は立憲民主党か国民民主党か…水面下の動きを解...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-19T08:34:11Z,484699,774.0,kdp
1,Mmd9PlMMcs0,【参院選CHECK】自民党には期待ゼロ！「外国人問題」を参政党･日本保守党だけでなく国民民主...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-05T08:00:25Z,64319,508.0,kdp
2,FXOb1Px5fsQ,【参院選CHECK】自民公明が内ゲバ状態！参政党躍進で公明党が大ピンチの愛知選挙区を須田解説！,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-17T08:40:19Z,351806,549.0,kdp
3,PL0PUpz1ah8,【参院選CHECK】内部ペーパー入手！自民党が過半数割れ確定でまさかの動きをはじめました！須...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-11T10:30:19Z,1103228,577.0,kdp
4,LBI5el11GPE,大敗濃厚の自民党！早くも選挙後の連立に向けて党内で動き始めているようです。特に気になるのは元...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-06T07:00:20Z,372353,565.0,kdp


=== Group4 (SSP) ===


,videoId,title,description,channelTitle,publishedAt,viewCount,duration,source
0,FXOb1Px5fsQ,【参院選CHECK】自民公明が内ゲバ状態！参政党躍進で公明党が大ピンチの愛知選挙区を須田解説！,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-17T08:40:19Z,351806,549.0,ssp
1,zS9XkP5flLA,【緊急速報】確定！自民公明の過半数割れ＆大阪選挙区で参政党が躍進！期日前投票の出口調査結果を...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-10T10:00:24Z,602221,485.0,ssp
2,Mmd9PlMMcs0,【参院選CHECK】自民党には期待ゼロ！「外国人問題」を参政党･日本保守党だけでなく国民民主...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-05T08:00:25Z,64319,508.0,ssp
3,UDgBZyGW1sY,【参院選CHECK】「参政党」を須田解説！躍進の理由は？神谷宗幣代表のメディア攻略成功、そこ...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-10T07:00:26Z,106467,711.0,ssp
4,DL2RwKgA0DU,【参院選速報】緊急の選対会議がヤバすぎる！極秘で木原選対委員長が大ピンチの静岡入り！独自取材...,ニュース解説を出来るだけ早く発信するチャンネル「撮って出しニュース」\n解説者：須田慎一郎\...,撮って出しニュース,2025-07-15T05:47:44Z,352401,342.0,ssp


In [ ]:
from google.colab import files

# 国民民主党 Group3
df_group3_kdp.to_csv("group3_kdp.csv", index=False, encoding="utf-8-sig")
files.download("group3_kdp.csv")

# 参政党 Group3
df_group3_ssp.to_csv("group3_ssp.csv", index=False, encoding="utf-8-sig")
files.download("group3_ssp.csv")

# 国民民主党 Group4
df_group4_kdp.to_csv("group4_kdp.csv", index=False, encoding="utf-8-sig")
files.download("group4_kdp.csv")

# 参政党 Group4
df_group4_ssp.to_csv("group4_ssp.csv", index=False, encoding="utf-8-sig")
files.download("group4_ssp.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>